In [ ]:
import pandas as pd
import sys
import os
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras import regularizers
from tensorflow.keras import layers
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.applications import EfficientNetB1
# from tensorflow.keras.applications import EfficientNetB2
# from tensorflow.keras.applications import EfficientNetB3
# from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import EfficientNetB5
# from tensorflow.keras.applications import EfficientNetB6
# from tensorflow.keras.applications import EfficientNetB7
from tensorflow import keras

In [ ]:
os.listdir('/kaggle/input/')

In [ ]:
# detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# from kaggle_datasets import KaggleDatasets
# GCS_DS_PATH_1 = KaggleDatasets().get_gcs_path('cassavaimagedata800x600')
# GCS_DS_PATH_2 = KaggleDatasets().get_gcs_path('efficientnet-h5')
# GCS_DS_PATH_3 = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')
# GCS_DS_PATH_4 = KaggleDatasets().get_gcs_path('model-k1')


In [ ]:
# ---------------------------------------------------------------------------------------------------
# パラメータ設定
# ---------------------------------------------------------------------------------------------------

labels = ["0", "1", "2", "3", "4"]          # ラベル設定
NUM_LABELS = len(labels)                    # 分類数
LEARNING_RATE = 0.0001                        # 学習率 : Adamで1e-2は大きすぎ、1e-3, 1e-4あたりが一般的
                                            # 自動減衰を実装したので大きめでOK
EPOCHS = 10                                 # エポック数
BATCH = 4                                   # バッチサイズ -> サンプル数の都合上、128はOK, 256以上はNG
# BATCH = 16 * tpu_strategy.num_replicas_in_sync
HEIGHT = 600                                # 画像の高さ
WIDTH = 800                                 # 画像の幅
HEIGHT_DEVIDE = 1                           # 画像サイズ縦(600/HEIGHT_DEVIDE)
WIDTH_DEVIDE  = 1                           # 画像サイズ横(800/WIDTH_DEVIDE)

In [ ]:
# 画像データをTFにぶち込める形式に変換
# ラベルはone-hotに
train_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = ImageDataGenerator(rescale=1./255)

train_images_path = '../input/cassavaimagedata800x600/train_images/train_images'
test_images_path  = '../input/cassavaimagedata800x600/test_images/test_images'

train_data = train_data_gen.flow_from_directory(train_images_path, 
                                                target_size=(int(WIDTH/WIDTH_DEVIDE), int(HEIGHT/HEIGHT_DEVIDE)),
                                                color_mode='rgb', 
                                                batch_size=BATCH,
                                                class_mode='categorical', 
                                                shuffle=True)

validation_data = val_data_gen.flow_from_directory(test_images_path, 
                                                   target_size=(int(WIDTH/WIDTH_DEVIDE), int(HEIGHT/HEIGHT_DEVIDE)),
                                                   color_mode='rgb', 
                                                   batch_size=BATCH,
                                                   class_mode='categorical', 
                                                   shuffle=True)

# 画像データとラベルを連ねていく感じ
#             batch, width, height, rgb 
# image_data      8,   800,    600,   3
#
#             betch, category
# label_data      8,     5
#

(image_data,label_data) = train_data.next()

In [ ]:
# with tpu_strategy.scope():
inputs = layers.Input(shape=(int(WIDTH/WIDTH_DEVIDE), int(HEIGHT/HEIGHT_DEVIDE), 3))
model = EfficientNetB5(include_top=False, 
                       input_tensor=inputs, 
                       weights="../input/efficientnet-h5/efficientnetb5_notop.h5",
                       classifier_activation="ReLU", # Softmaxだと緩やかに発散
                       pooling='max')


# Rebuild top
# x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
# x = layers.BatchNormalization()(x)
x = layers.Dense(units = BATCH, name="avg_pool")(model.output)
top_dropout_rate = 0.25
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(NUM_LABELS, 
                       activation='softmax',
                       name="pred")(x)


model = keras.Model(inputs, outputs, name="EfficientNet")

# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'],
#                   steps_per_execution=32 # TPU
             )


model.summary()

In [ ]:
# Early Stopping
# epoch少なくしてるときはいらない
early_stopping = EarlyStopping(
                        monitor   = 'val_loss',
                        min_delta = 0.0,
                        patience  = 10)

# 2epoch改善が見られなければLearning_rateを0.5倍
reduce_lr = ReduceLROnPlateau(
                        monitor  = 'val_loss',
                        factor   = 0.5,
                        patience = 2,
                        min_lr   = 0.0001)

hist = model.fit(train_data, 
                 epochs=EPOCHS, 
                 validation_data=validation_data, 
                 verbose=1,
                 callbacks=[early_stopping, reduce_lr]
                )

In [ ]:
model.save('model.h5', include_optimizer=False)
# model = keras.models.load_model('model.h5', compile=False)

# TPU
# save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
# model.save('./model', options=save_locally) # saving in Tensorflow's "SavedModel" format

In [ ]:
print("Done.")

# Output作成

In [ ]:
model = keras.models.load_model('../input/model-k1/model.h5', compile=False)
# TPU
# with strategy.scope():
#     load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
#     model = tf.keras.models.load_model('./model', options=load_locally) # loading in Tensorflow's "SavedModel" format

In [ ]:
output_csv = '../input/cassava-leaf-disease-classification/sample_submission.csv'
submission = pd.read_csv(output_csv)
submission.head()

In [ ]:
output_img = '../input/cassava-leaf-disease-classification/test_images/2216849948.jpg'
img = load_img(output_img)
img = img_to_array(img)
img = tf.reshape(img, (-1, int(WIDTH/WIDTH_DEVIDE), int(HEIGHT/HEIGHT_DEVIDE), 3))
pred = model.predict(img)
pred = np.argmax(pred)


In [ ]:
pred

In [ ]:
submission_result = pd.DataFrame({'image_id' : submission.image_id, 'label' : pred})
submission_result.to_csv('submission.csv', index=False)